# Тренировка

In [ ]:
import os
from datasets import load_dataset

data_dir = os.path.join(os.pardir, 'data')
dataset = load_dataset('csv', data_files={
    'train': os.path.join(data_dir, 'train.csv'),
    'test': os.path.join(data_dir, 'test.csv')
})

train_dataset = dataset['train']
test_dataset = dataset['test']

/Users/vovazakharov/anaconda3/envs/shad/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import AutoTokenizer, MambaForCausalLM
from transformers import DataCollatorForLanguageModeling

model_id = "state-spaces/mamba-130m-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def preprocess_function(examples):
    texts = [f"Translate {type} differential equation: {equation}. Solution: {answer}" 
             for equation, answer, type in zip(examples['equation'], examples['answer'], examples['type'])]
    
    tokenized = tokenizer(texts, truncation=True, max_length=512, padding="max_length")
    return tokenized

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

model = MambaForCausalLM.from_pretrained(model_id)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False 
)

/Users/vovazakharov/anaconda3/envs/shad/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/vovazakharov/anaconda3/envs/shad/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <5AA8DD3D-A2CC-31CA-8060-88B4E9C18B09> /Users/vovazakharov/anaconda3/envs/shad/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <8B7B82B6-E557-3363-B7AC-84C29F398EBA> /Users/vovazakharov/anaconda3/envs/shad/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Map: 100%|██████████| 15235/15235 [00:01<00:00, 8526.45 examples/s]
The fast path is not available because one of `(selective_sta

In [ ]:
import torch
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./mamba_diffeq",
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    fp16=True if torch.cuda.is_available() else False,
    gradient_accumulation_steps=4,
    report_to="tensorboard"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 12.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 14.3 MB/s eta 0:00:00 0:00:01


/Users/vovazakharov/anaconda3/envs/shad/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

trainer.save_model("./mamba_diffeq")
tokenizer.save_pretrained("./mamba_diffeq")

# Инференс

In [ ]:
from transformers import pipeline

translator = pipeline(
    "text-generation",
    model="./mamba_diffeq",
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

input_eq = "Translate polynomial differential equation: dy/dx = x^2. Solution: "
generated = translator(
    input_eq,
    max_length=200,
    num_return_sequences=1,
    temperature=0.7,
    top_p=0.9
)

print(generated[0]['generated_text'])